                        llama3-8b  pass@1: 0.38 | phi3 pass@1: 0.60  [HumanEval]

In [2]:
from data import CodeCompletionGenerator, load_config
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm as tqdm


config_name = "llama3_8b_instruct"
code_gen = CodeCompletionGenerator.from_config(config_name, device="mps")

# Get the error message on sub-batch and do fast adaptation 
system_prompt = "Do not fuck it up"
# Run Inference on HumanEval | Generate -> GetError -> SaveErro
indices = [55]
code_gen.run_human_eval_test(indices=indices, global_system_prompt=system_prompt)


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

original/params.json:   0%|          | 0.00/211 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Generating code completions:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: [INSTURCTION] Do not fuck it up [\INSTRUCTION] 


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """

    if n <= 1:
        return n
    else:
        return fib(n-1) + fib(n-2)


def test_fib():
    assert fib(1) == 1
    assert fib(2) == 1
    assert fib(3) == 2
    assert fib(4) == 3
    assert fib(5) == 5
    assert fib(6) == 8
    assert fib(7) == 13
    assert fib(8) == 21
    assert fib(9) == 34
    assert fib(10) == 55
    print("All tests pass.")


test_fib()
print(fib(10))  # Output: 55
print(fib(20))  # Output: 6765
print(fib(30))  # Output: 832040
print(fib(40))  # Output: 102334155
print(fib(50))  # Output: 12586269025
print(fib(60))  # Output: 1548008755
print(fib(70))  # Output: 196418
print(fib(80))  # Output: 317811
print(fib(90))  # Output: 832040
print(fib(100))  # Output: 354224848179261915075
print(fib(110))  # Output: 109461993441772790772930
print(fib(120))  # Output: 1655801411

Generating code completions: 100%|██████████| 1/1 [00:23<00:00, 23.17s/it]


Prompt: 239.622 tokens-per-sec
Generation: 22.309 tokens-per-sec
Writing error information to data/log/llama3_8b_instruct-55__info.jsonl...


100%|██████████| 1/1 [00:10<00:00, 10.03s/it]


In [4]:
from human_eval.data import write_jsonl, read_problems, stream_jsonl, HUMAN_EVAL

problem_dict = read_problems()

'    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    return fib(n - 1) + fib(n - 2)\n'

#### Synthetic Dataset Writer w. DataDreamer
* Unfortunately it's not that reliable .....

In [5]:
# It's important to collect Toy case to check the performances here
import torch
import transformers

import pyreft

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, torch_dtype=torch.bfloat16, device_map=device)

# get tokenizer
model_max_length = 2048
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=model_max_length, 
    padding_side="right", use_fast=False)
if "Meta-Llama-3-" in model_name_or_path:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
else:
    tokenizer.pad_token = tokenizer.unk_token

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

system_prompt = "You are a helpful assistant."


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from human_eval.data import write_jsonl, read_problems, stream_jsonl, HUMAN_EVAL

problem_dict = read_problems()
problem = problem_dict['HumanEval/55']
prompt = problem['prompt']
canonical_solution = problem['canonical_solution']
canonical_solution
training_examples = [[prompt, canonical_solution]]

In [8]:
# get reft model
reft_config = pyreft.ReftConfig(representations=[{
    "layer": l, "component": "block_output",
    "low_rank_dimension": 2,
    "intervention": pyreft.LoreftIntervention(embed_dim=model.config.hidden_size,
    low_rank_dimension=2)} for l in [8, 16, 24]])
reft_model = pyreft.get_reft_model(model, reft_config)
reft_model.set_device("cpu")
reft_model.print_trainable_parameters()

trainable intervention params: 49,158 || trainable model params: 0
model params: 8,030,269,440 || trainable%: 0.00061215878704065


In [9]:
# Fast Adaptation on a toy training dataset 
import pyreft

# position info about the interventions
share_weights = True # whether the prefix and suffix interventions sharing weights.
positions="f1+l1"    # the intervening positions of prefix tokens (f[irst]1) and suffix tokens (l[ast]1).
first_n, last_n = pyreft.parse_positions(positions)

data_module = pyreft.make_multiple_position_supervised_data_module(
    tokenizer, model, [tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt}, 
            {"role": "user", "content": e[0]}
        ], tokenize=False
    ) for e in training_examples], [
        tokenizer.apply_chat_template(
            [{"role": "assistant", "content": e[1]}], tokenize=False,
        )[len(tokenizer.bos_token):] for e in training_examples
    ], 
    positions=positions, num_interventions=len(reft_config.representations), share_weights=share_weights, nonstop=False)

In [10]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=100.0, output_dir="./tmp", per_device_train_batch_size=10, 
    learning_rate=4e-3, logging_steps=20)
trainer = pyreft.ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, **data_module)
_ = trainer.train()

/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


TypeError: BFloat16 is not supported on MPS

In [14]:
indices = [55]
system_prompt = "if n==1: return 1"
code_gen.run_human_eval_test(indices=indices, global_system_prompt=system_prompt)

Generating code completions:   0%|          | 0/1 [00:00<?, ?it/s]

Prompt: <s><|user|>
You are a friendly chatbot. [INSTRUCTION] if n==1: return 1 [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>

```python

def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n == 1:
        return 1
    elif n == 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

# Note: The above implementation is not efficient for large values of n due to its exponential time complexity. For large values of n, it's better to use dynamic programming or memoization techniques to optimize the function.

```<|end|><|assistant|> ```python

def fib(n: int):
    """
    Return the n-th Fibonacci numbe

Generating code completions: 100%|██████████| 1/1 [00:12<00:00, 12.61s/it]


Prompt: 893.454 tokens-per-sec
Generation: 41.131 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55__info.jsonl...


100%|██████████| 1/1 [00:00<00:00, 23.91it/s]


#### Fast Adaptation || Monte Carlo Tree Search || Case Study (What is an effective way of improving codeing capacity ?)

In [4]:
# Case Study Here
from src.fast_adapt import *
from src.prompt import *
import numpy as np

indices = [55] # Pin-down this Fibonacci sequence calculation issue and solve the case study
init_prompt = "Do not fuck it up"
id = 4527
node = Node.from_prompt(indices, init_prompt, id, code_gen)


Generating code completions:   0%|          | 0/1 [00:00<?, ?it/s]

System Prompt Line 174:  Do not fuck it up
Prompt: <s><|user|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

```<|end|><|assistant|> ```python

def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    ""

Generating code completions: 100%|██████████| 1/1 [00:24<00:00, 24.91s/it]


Prompt: 772.389 tokens-per-sec
Generation: 20.683 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55_4527_info.jsonl...


100%|██████████| 1/1 [00:00<00:00, 22.02it/s]


In [17]:
node.infos[0]['canonical_solution']

'    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    return fib(n - 1) + fib(n - 2)\n'

In [16]:
one_shot_prompt = node.infos[0]['canonical_solution'][:20]
new_node = Node.from_prompt(indices, one_shot_prompt, id, code_gen)

Generating code completions:   0%|          | 0/1 [00:00<?, ?it/s]

System Prompt Line 174:      if n == 0:
     
Prompt: <s><|user|>
You are a friendly chatbot. [INSTRUCTION]     if n == 0:
      [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        a, b = 0, 1
        for _ in range(n-1):
            a, b = b, a + b
        return b

```<|end|><|assistant|> ```python

def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n == 0:
        return 0
    elif n == 1:
        retu

Generating code completions: 100%|██████████| 1/1 [00:12<00:00, 12.59s/it]


Prompt: 823.153 tokens-per-sec
Generation: 41.249 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55_4527_info.jsonl...


100%|██████████| 1/1 [00:00<00:00, 24.43it/s]


In [6]:
max_search = 1
# Initialize Sub-Batch Indices
init_prompt = "Do not fuck it up"
# Initialize the MCTS object
mcts = MCTS(init_prompt, 0, indices, code_gen, max_search=max_search)

# Run the search loop
while not mcts.is_search_complete():
    node_to_expand = mcts.select_node()
    mcts.expand_node(node_to_expand)


Prompt: <s><|user|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b


This function now correctly computes the n-th Fibonacci number.<|end|><|assistant|> The provided solution correctly implements the Fibonacci sequence calculation. The function starts by handling edge cases where the input is less than or equal to zero. It then returns the first two Fibonacci numbers (0 and 1) for the first two inputs. For all other cases, it uses a loop to calculate the n-th Fibonacci number by iteratively updating two variables that ho

Generating code completions: 100%|██████████| 1/1 [00:07<00:00,  7.89s/it]


Prompt: 261.604 tokens-per-sec
Generation: 42.785 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55_0_info.jsonl...


100%|██████████| 1/1 [00:00<00:00, 25.78it/s]


Prompt: <s><|user|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be positive integer"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b


This function now correctly calculates the n-th Fibonacci number. It uses an iterative approach, which is more efficient than the recursive approach, especially for large values of n.<|end|><|assistant|> The provided solution is indeed correct and efficient for calculating the n-th Fibonacci number. The function starts by handling edge cases (n=0 and n=1) and then proceeds to calculate the Fibonacci number iteratively. This approach ensures that the func

...

Prompt: <s><|user|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be positive integer"
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

# Testing the function
assert fib(10) == 55
assert fib(1) == 0
assert fib(8) == 21
print("All tests passed successfully.")<|end|><|assistant|> Your implementation of the Fibonacci sequence is correct and well-documented. The function handles edge cases, such as when `n` is less than or equal to 0, by returning a message indicating the input should be a positive integer.

The function efficiently calculates the n-th Fibonacci number using an iterative appr

Generating code completions: 100%|██████████| 1/1 [00:08<00:00,  8.27s/it]

Prompt: 872.464 tokens-per-sec
Generation: 42.618 tokens-per-sec


Writing error information to data/log/phi_3_instruct-55_1_info.jsonl...


100%|██████████| 1/1 [00:00<00:00, 26.76it/s]


In [9]:
mcts.get_best_prompt()

'Do not fuck it up'

In [10]:
mcts.best_node.infos

[{'task_id': 'HumanEval/55',
  'completion': '    """Return n-th Fibonacci number.\n    >>> fib(10)\n    55\n    >>> fib(1)\n    1\n    >>> fib(8)\n    21\n    """\n    if n <= 0:\n        return "Input should be positive integer."\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n        for _ in range(2, n):\n            a, b = b, a + b\n        return b',
  'prompt': 'Do not fuck it up',
  'entry_point': 'fib',
  'canonical_solution': '    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    return fib(n - 1) + fib(n - 2)\n',
  'program': '\n\ndef fib(n: int):\n    """Return n-th Fibonacci number.\n    >>> fib(10)\n    55\n    >>> fib(1)\n    1\n    >>> fib(8)\n    21\n    """\n    """Return n-th Fibonacci number.\n    >>> fib(10)\n    55\n    >>> fib(1)\n    1\n    >>> fib(8)\n    21\n    """\n    if n <= 0:\n        return "Input should be positive integer."\n    elif n == 1:\n        return 0\n    elif n == 2:\

In [ ]:
# Compression based solution --> Start with 1 shot prompting, then gradually compress the prompt while maintaining the performance



In [2]:
# Reflective Adaptation | Use Groq for fast iteration | CUDA device could also be fast (?)
from src.fast_adapt import *
from src.prompt import *
import numpy as np

batches = [indices.tolist() for indices in np.array_split(list(range(162)), 60)]
max_search = 30
pb = tqdm(total=len(batches), desc="MCTS Search Batches")
for i, indices in enumerate(batches):
    pb.desc = "MCTS Search Batches: %d/%d" % (i + 1, len(batches))
    # Initialize Sub-Batch Indices
    init_prompt = "Do not fuck it up"
    # Initialize the MCTS object
    mcts = MCTS(init_prompt, 0, indices, code_gen, max_search=max_search)

    # Run the search loop
    while not mcts.is_search_complete():
        node_to_expand = mcts.select_node()
        mcts.expand_node(node_to_expand)
        pb.update(1)
    pb.n = max_search * (i + 1)
    # Retrieve the best prompt after the search is complete
    best_prompt = mcts.get_best_prompt()
    print("Best Prompt on Batch %d: %s" % (i, best_prompt))
    mcts.save_best_node()
    # break

MCTS Search Batches:   0%|          | 0/60 [00:00<?, ?it/s]

Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
<|end|>
<|assistant|>

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in

Prompt: 1253.196 tokens-per-sec
Generation: 40.615 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>
from typing import List


def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """
<|end|>
<|assistant|>

from typing import List

def separate_paren_groups(paren_string: str) -> List[str]:
    # Remove spaces from t

Prompt: 1198.718 tokens-per-sec
Generation: 40.083 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """
<|end|>
<|assistant|>


def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.


Generating code completions: 100%|██████████| 3/3 [00:27<00:00,  9.15s/it]


Prompt: 998.294 tokens-per-sec
Generation: 41.280 tokens-per-sec
Writing error information to data/log/phi_3_instruct-0-1-2_0_info.jsonl...


100%|██████████| 3/3 [00:00<00:00, 25.26it/s]


AttributeError: 'MCTS' object has no attribute 'root_node'